# Demo of the Pillar Assessment Tool

### 0. Setup

First import neccissary modules from `scripts\climate_policy_pipelines\cp1`

In [1]:
# Import necessary modules
import sys
import os
from pathlib import Path

# Get the absolute path of the project root directory
notebook_dir = Path(os.getcwd())  
project_root = notebook_dir.parent.parent  # Go up TWO levels instead of one

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"Added {project_root} to sys.path")

Added /Users/jessiefung/Desktop/DS205/group-6-final-project to sys.path


In [2]:
from scripts.climate_policy_pipelines.cp1.pipeline import run_cp1a_assessment
from scripts.climate_policy_pipelines.cp1.pipeline import run_cp1a_assessment_large_context
from scripts.climate_policy_pipelines.cp1.pipeline import run_cp1b_assessment

/Users/jessiefung/Desktop/DS205/group-6-final-project/.venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from scripts.climate_policy_pipelines.cp1.prompts import cp1a_criterion_1_prompt
from scripts.climate_policy_pipelines.cp1.prompts import cp1a_final_assessment_prompt
from scripts.climate_policy_pipelines.cp1.prompts import comprehensive_assessment_prompt

# CP1a Asessment

### 1. What content does it use?

Lets have a look at the chunks it retrieves in order to inform its assessment:

In [4]:
cp1a_1_content = [
  {
    "chunk_content": "The UK’s 2035 Nationally Determined Contribution (NDC) target, announced by the Prime Minister at COP29 in November 2024, is to reduce all greenhouse gas emissions by at least 81% on 1990 levels, excluding international aviation and shipping emissions.",
    "document": "UK’s 2035 Nationally Determined Contribution (NDC) emissions reduction target under the Paris Agreement - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The UK’s 2035 Updated Nationally Determined Contribution confirms the topline ambition of reducing emissions by 81% (based on 1990 levels). This target is in line with advice given by the Climate Change Committee in October 2024, and represents a 1.5°C-aligned, economy-wide absolute emissions reduction target.",
    "document": "The UK's NDC and the Seventh Carbon Budget - Chapter Zero",
    "page_number": 1
  },
  {
    "chunk_content": "The UK government has been advised by the independent Climate Change Committee (CCC) to commit to reducing greenhouse gas emissions by 81% by 2035 compared to 1990 levels. This recommendation aims to be both fair and ambitious in contributing to the Paris Agreement.",
    "document": "UK must raise its climate ambition to restore global reputation, says adviser - Financial Times",
    "page_number": 1
  },
  {
    "chunk_content": "At COP29 in Baku, British PM Keir Starmer announced an ambitious target to reduce the UK's greenhouse gas emissions by at least 81% by 2035, compared to 1990 levels. This is an increase from the previous target of a 68% cut by 2030, with the ultimate goal of achieving carbon neutrality by 2050.",
    "document": "COP29: British PM Starmer wants to regain 'climate leadership' - Le Monde",
    "page_number": 1
  },
  {
    "chunk_content": "The UK submitted its 2035 NDC to the UNFCCC on 30 January 2025. It will also be available on the UNFCCC NDC registry. The UK will produce an updated cross-economy plan in due course, outlining the policies needed to deliver carbon budgets 4-6 and the 2030 and 2035 NDCs on the pathway to net zero by 2050.",
    "document": "UK's 2035 Nationally Determined Contribution (NDC) emissions reduction target under the Paris Agreement - GOV.UK",
    "page_number": 1
  }
]

cp1a_2_content = [
  {
    "chunk_content": "The Climate Change Act 2008 was the first law of its kind. It makes long-term emissions reduction targets legally-binding, opening the door to citizen lawsuits if the government misses them, and introduces interim targets or 'carbon budgets' to keep the UK on track.",
    "document": "Climate Legislation in the United Kingdom - Canadian Climate Institute",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Act, as amended in 2019, commits the UK to ‘net zero’ by 2050. The original act, passed in 2008, committed the UK to an 80% reduction of greenhouse gas emissions by 2050, compared to 1990 levels.",
    "document": "Climate change targets: the road to net zero? - House of Lords Library",
    "page_number": 1
  },
  {
    "chunk_content": "In June 2021, the government set in law the sixth carbon budget (CB6) limiting the volume of greenhouse gases emitted from 2033 to 2037. CB6 reduces emissions by approximately 78% by 2035 compared to 1990 levels.",
    "document": "Why Net Zero - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The UK Parliament passed The Climate Change Act 2008 (2050 Target Amendment) Order 2019, which amends Section 1 of the Climate Change Act 2008 to require the UK to reduce its GHG emissions by 100% compared to their level in 1990.",
    "document": "UK Becomes First G7 Country to Legislate for 'Net-Zero' Emissions - WilmerHale",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Act (2008) made the UK the first country to establish a long-term legally binding framework to cut carbon emissions. It contains a target requiring emissions reductions to Net Zero by 2050.",
    "document": "Climate action - Climate Change Committee",
    "page_number": 1
  }
]

cp1a_3_content = [
  {
    "chunk_content": "The UK’s Net Zero Strategy outlines a comprehensive plan to achieve net zero greenhouse gas emissions by 2050, setting legally binding carbon budgets and detailing sector-specific policies to reduce emissions.",
    "document": "Net Zero Strategy: Build Back Greener - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Act 2008 mandates the UK government to set legally binding carbon budgets, which cap the amount of greenhouse gases emitted over a five-year period, ensuring progress towards the 2050 net zero target.",
    "document": "Climate Change Act 2008 - legislation.gov.uk",
    "page_number": 1
  },
  {
    "chunk_content": "The UK Emissions Trading Scheme (UK ETS), established in 2021, sets a cap on total greenhouse gas emissions and allows trading of emission allowances, providing a market-based approach to incentivize emission reductions.",
    "document": "UK Emissions Trading Scheme - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The Renewables Obligation requires electricity suppliers to source an increasing proportion of electricity from renewable sources, supporting the development of renewable energy technologies through financial incentives.",
    "document": "Renewables Obligation (United Kingdom) - Wikipedia",
    "page_number": 1
  },
  {
    "chunk_content": "The Climate Change Levy is a tax on energy delivered to non-domestic users in the UK, designed to encourage energy efficiency and reduce carbon emissions in the industrial, commercial, and public services sectors.",
    "document": "Climate Change Levy - GOV.UK",
    "page_number": 1
  }
]

cp1a_4_content = [
  {
    "chunk_content": "The Environment Act 2021 provides a broad framework through which specific environmental rules and regulation will operate. While it remains to be seen how the government will implement the Act’s provisions, it represents a significant opportunity for environmental law to drive the UK’s transition to net zero. The Act requires the Government to devise long-term legally binding targets concerning biodiversity, water, air quality, resource efficiency and waste management.",
    "document": "Environmental Law and Climate Change - Centre for Climate Engagement",
    "page_number": 1
  },
  {
    "chunk_content": "The Government’s 25-year environmental plan supports the Act by requiring that all policies, programmes, and investment decisions take due consideration of climate change.",
    "document": "Environmental Law and Climate Change - Centre for Climate Engagement",
    "page_number": 1
  },
  {
    "chunk_content": "The Environment Act 2021, passed in the UK, sets forth new environmental targets and establishes the Office for Environmental Protection (OEP) as an oversight body. It outlines a governance framework and mandates an environmental improvement plan for England and Wales, focusing on enhancing environmental standards and biodiversity.",
    "document": "British environmental law - Wikipedia",
    "page_number": 1
  },
  {
    "chunk_content": "The UK’s Net Zero Strategy outlines a comprehensive plan to achieve net zero greenhouse gas emissions by 2050, setting legally binding carbon budgets and detailing sector-specific policies to reduce emissions.",
    "document": "Net Zero Strategy: Build Back Greener - GOV.UK",
    "page_number": 1
  },
  {
    "chunk_content": "The Net Zero Strategy builds on the Government’s 10 point plan for a green industrial revolution which was published on 18 November 2020. It focuses on advancing offshore wind, driving the growth of low carbon hydrogen, delivering new and advanced nuclear power, accelerating the shift to zero emission vehicles, and other key areas.",
    "document": "Government support for a nature and climate declaration - House of Commons Library",
    "page_number": 1
  }
]

cp1a_retrieved_chunks = [cp1a_1_content, cp1a_2_content, cp1a_3_content, cp1a_4_content]


### 2. Lets have a look at the prompts and workflow used:

CP1a is defined as: **Does the country have a framework climate law or equivalent?**

Individualy test the following (A country is assessed as ‘Yes’ if it has a framework climate law that fulfils all of
the following):

ASCOR brokes down CP1a into 4 criteria, which a different LLM handles:

1. It sets a strategic direction for decarbonisation

2. It is enshrined in law

3. It sets out at least one of the following obligations

4. Also check this In exceptional cases, the combination of a broad environmental law and a clearly linked executive climate strategy may be sufficient to meet these criteria

We can see the prompt for criteria 1 here:

In [5]:
# Print the system message template
print("System Message Template:")
print(cp1a_criterion_1_prompt.messages[0].prompt.template)

print("\n" + "="*50 + "\n")

# Print the human message template  
print("Human Message Template:")
print(cp1a_criterion_1_prompt.messages[1].prompt.template)

System Message Template:
You are an expert legal analyst specializing in climate legislation. 
    Your task is to evaluate whether a climate law sets a strategic direction for decarbonisation.
    
    A law meets this criterion if it includes a clear statement to meet the goals of the Paris Agreement 
    OR a national long-term decarbonisation target.
    
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

     
    Respond with only 'YES' or 'NO' followed by a brief explanation.


Human Message Template:
Context: {context}

Does this law set a strategic direction for decarbonisation?


According to ASCOR if 1,2,3 are satisfied or 4 is satisfied, then CP1a is answered as Yes.


We can therefore see the prompt for the model that determines the overall assessment: 

In [6]:
# Print the system message template
print("System Message Template:")
print(cp1a_final_assessment_prompt.messages[0].prompt.template)

print("\n" + "="*50 + "\n")

# Print the human message template  
print("Human Message Template:")
print(cp1a_final_assessment_prompt.messages[1].prompt.template)

System Message Template:
You are an expert legal analyst making a final assessment of climate legislation.
    
    A country is assessed as 'YES' for having framework climate law if:
    - Criteria 1, 2, AND 3 are all satisfied, OR
    - Criterion 4 is satisfied (exceptional case)
   
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    Based on the individual assessments, provide a final 'YES' or 'NO' answer with reasoning.


Human Message Template:
Individual criterion assessments:
    Criterion 1 (Strategic direction): {criterion_1_result}
    Criterion 2 (Enshrined in law): {criterion_2_result}
    Criterion 3 (Obligations): {criterion_3_result}
    Criterion 4 (Exceptional case): {criterion_4_result}
    
    What is the final assessment?


### 3. Demo assessment 

Lets run an assessment for (country)

In [7]:
run_cp1a_assessment(country="ALB")

Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You 

AIMessage(content='Based on the individual criterion assessments, the final assessment is: **YES**\n\nThe country satisfies Criteria 1, 2, and 3, which are all required for a \'YES\' assessment. Specifically:\n\n* Criterion 1 (Strategic direction) is satisfied, as the law includes a statement to fulfill reporting requirements under the UNFCCC and the Paris Agreement, and mentions a national long-term decarbonisation target (page not specified, Albania’s National Adaptation Plan First - progress report and National Energy and Climate Plan 2019 Draft).\n* Criterion 2 (Enshrined in law) is satisfied, as climate legislation is enshrined in law through the "Law for Climate Change" (page not specified, National Energy and Climate Plan 2019 Draft).\n* Criterion 3 (Obligations) is satisfied, as the law sets out specific obligations, such as developing policy instruments and public spending in support of climate goals (page not specified, National Energy and Climate Plan 2019 Draft).\n\nAdditio

# CP1a Large Context Window Asessment

Another tool just uses a more powerful large context window LLM to assess CP1a. Here is the prompt:

In [8]:
# Print the system message template
print("System Message Template:")
print(comprehensive_assessment_prompt.messages[0].prompt.template)

print("\n" + "="*50 + "\n")

# Print the human message template  
print("Human Message Template:")
print(comprehensive_assessment_prompt.messages[1].prompt.template)

System Message Template:
You are an expert legal analyst specializing in climate legislation assessment. 

Your task is to evaluate whether a country has a framework climate law based on specific criteria and provide a structured markdown assessment.

EVALUATION CRITERIA:
A country is assessed as 'YES' if it has a framework climate law that fulfils ALL of criteria 1, 2, AND 3, OR criterion 4:

1. STRATEGIC DIRECTION: Sets a strategic direction for decarbonisation (must include a clear statement to meet the goals of the Paris Agreement OR a national long-term decarbonisation target)

2. ENSHRINED IN LAW: Is enshrined in law (must be legislative rather than executive, except in particular political systems)

3. OBLIGATIONS: Sets out at least one of the following obligations:
   - Meeting a national target
   - Developing, revising, implementing or complying with domestic plans, strategies or policies
   - Developing policy instruments such as regulation, taxation or public spending in su

Lets see how it performs on the same country:
`

In [10]:
run_cp1a_assessment_large_context(country="ALB")


Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You 

Large Context Assessment:
# Climate Legislation Assessment: CP 1.a Framework Climate Law

## Individual Criterion Evaluation

### Criterion 1: Strategic Direction for Decarbonisation
**Result:** YES
**Reasoning:** The National Energy and Climate Plan 2019 Draft mentions that "Setting up the ETS in Albania will contribute to reach the NDC target for reduction of GHG emission from the industry sector as well as the NECP target for decarbonisation of the economy" (page not specified, doc National Energy and Climate Plan 2019 Draft). Additionally, the document mentions that the country aims to "fulfill the reporting requirements under the United Nations Framework Convention on Climate Change and the Paris Agreement" (page not specified, doc Albania’s National Adaptation Plan First - progress report). This suggests that the country has a strategic direction for decarbonisation aligned with the Paris Agreement.

### Criterion 2: Enshrined in Law
**Result:** YES
**Reasoning:** The National En

AIMessage(content='# Climate Legislation Assessment: CP 1.a Framework Climate Law\n\n## Individual Criterion Evaluation\n\n### Criterion 1: Strategic Direction for Decarbonisation\n**Result:** YES\n**Reasoning:** The National Energy and Climate Plan 2019 Draft mentions that "Setting up the ETS in Albania will contribute to reach the NDC target for reduction of GHG emission from the industry sector as well as the NECP target for decarbonisation of the economy" (page not specified, doc National Energy and Climate Plan 2019 Draft). Additionally, the document mentions that the country aims to "fulfill the reporting requirements under the United Nations Framework Convention on Climate Change and the Paris Agreement" (page not specified, doc Albania’s National Adaptation Plan First - progress report). This suggests that the country has a strategic direction for decarbonisation aligned with the Paris Agreement.\n\n### Criterion 2: Enshrined in Law\n**Result:** YES\n**Reasoning:** The National

# CP1b Assessment

CP1b is defined by ASCOR as: **Does the country’s framework climate law specify key accountability elements?**


A country is assessed as ‘Yes’ if its framework climate law contains all three of the following accountability elements: 
1. Specification of who is accountable to whom for at least one stated obligation (e.g. accountability of executive to parliament, or private parties to executive authorities) 
2. Specification of how compliance is assessed for at least one stated obligation (e.g. transparency mechanisms in the form of monitoring, reporting and verification, parliamentary oversight, expert assessments, court proceedings) 
3. Specification of what happens in the case of non-compliance for at least one stated obligation (e.g. parliamentary intervention, judicial orders, financial penalties). 

Like with CP1a, these are evaluated separately and then evaluted together by a evaluator LLM. There is additional guidance in the methodology on how to asses these criteria wich is included in the prompts. 

Lets see how the tool performs:

In [11]:
run_cp1b_assessment(country="ALB")

Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You 

Detailed Assessment:
Based on the individual criterion assessments, the final assessment is:

**NO**

Reasoning: Although the country's framework climate law meets Criteria 1 and 2, it does not meet Criterion 3, which requires specification of what happens in the case of non-compliance for at least one stated obligation. Since the documents provided do not mention any consequences or actions to be taken in case of non-compliance, the country's framework climate law does not fully meet the accountability requirements.


AIMessage(content="Based on the individual criterion assessments, the final assessment is:\n\n**NO**\n\nReasoning: Although the country's framework climate law meets Criteria 1 and 2, it does not meet Criterion 3, which requires specification of what happens in the case of non-compliance for at least one stated obligation. Since the documents provided do not mention any consequences or actions to be taken in case of non-compliance, the country's framework climate law does not fully meet the accountability requirements.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 555, 'total_tokens': 652, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'system_fingerprint': None, 'id': 'chatcmpl-fac61a06d7574f8284957e8dc041ae8a', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--6b31214b-f45e-4d45-b013-4a82b9fee599-0', usage_metadata=